In [9]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
import sys
from os import listdir

In [10]:
def create_paper_summary_data_frame():
    doi = input("DOI for paper: ")
    human_src_id = input("Human Src ID: ")
    directory = input("Directory for paper: ")
    
    #list of all the [folders/plots] for paper ()
    plots_in_dir = !ls $directory 
    num_plots = len(plots_in_dir)
    
    for i in range(num_plots):
        paper_descript = input("Describe some key aspects about the paper ")
        plot_foldername = directory + "/" + plots_in_dir[i]
        
        # list of all the csv's associated with plot
        data_segs_for_plot = !ls $plot_foldername 
        print("This is plot number " + str(i))
        num_segs_in_plot = len(data_segs_for_plot)
        

        
        cols = ['Air Vel [cm/s]','FSR [mm/s]','d_c [mm]', 'd_i [mm]',
                      'P_atm', '[O2]', 'g', 'q_ext [kW/m^2]', 'core material', 
                      'insulation material','orientation (vert/horiz)', 'up | down', 'Human Src ID', 'DOI', 'Plot #']
        final_df = pd.DataFrame(columns = cols)
        x_value, y_value = 0, 1
        for j in range(1):#num_segs_in_plot):
            name_of_seg_in_plot = plot_foldername + "/" + data_segs_for_plot[j]
            print(name_of_seg_in_plot)
            skeleton_row = []
            for k in cols:
                if k == 'Human Src ID':
                    skeleton_row.append(human_src_id)
                    continue
                if k == 'DOI':
                    skeleton_row.append(doi)
                    continue
                if k == 'Plot #':
                    skeleton_row.append(i)
                    continue
                val = input("For data seg " + data_segs_for_plot[j] + " " + k + " ")
                if (str(val) == 'x'):
                    x_value = cols.index(k)
                    skeleton_row.append(0)
                elif (str(val) == 'y'):
                    y_value = cols.index(k)
                    skeleton_row.append(0)
                elif k in ['core material', 'insulation material', 'orientation (vert/horiz)', 'up | down']:
                    skeleton_row.append(str(val or 'NA'))
                else: 
                    skeleton_row.append(float(val or 'nan'))
            csv_data = pd.read_csv(name_of_seg_in_plot, skiprows=5, index_col=False)
            skeleton_df = pd.DataFrame(np.array([skeleton_row]), columns=cols)
            skeleton_df = pd.concat([skeleton_df]*csv_data.shape[0], ignore_index=True)
            csv_data.columns = [0,1]
            skeleton_df[cols[x_value]], skeleton_df[cols[y_value]] = csv_data[0], csv_data[1]
            final_df = final_df.append(skeleton_df, ignore_index=True)
            final_df
        return final_df

In [11]:
data = create_paper_summary_data_frame()


DOI for paper: 123123
Human Src ID: Urban
Directory for paper: kobayashi
Describe some key aspects about the paper asd
This is plot number 0
kobayashi/1/8mm_Cu.csv
For data seg 8mm_Cu.csv Air Vel [cm/s] x
For data seg 8mm_Cu.csv FSR [mm/s] y
For data seg 8mm_Cu.csv d_c [mm] 1
For data seg 8mm_Cu.csv d_i [mm] 3
For data seg 8mm_Cu.csv P_atm 100
For data seg 8mm_Cu.csv [O2] 
For data seg 8mm_Cu.csv g 
For data seg 8mm_Cu.csv q_ext [kW/m^2] 
For data seg 8mm_Cu.csv core material 
For data seg 8mm_Cu.csv insulation material 
For data seg 8mm_Cu.csv orientation (vert/horiz) 
For data seg 8mm_Cu.csv up | down 


In [3]:
import pandas as pd
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.sampledata.periodic_table import elements

In [5]:
elements

,atomic number,symbol,name,atomic mass,CPK,electronic configuration,electronegativity,atomic radius,ion radius,van der Waals radius,...,EA,standard state,bonding type,melting point,boiling point,density,metal,year discovered,group,period
0,1,H,Hydrogen,1.00794,#FFFFFF,1s1,2.20,37.0,NaN,120.0,...,-73.0,gas,diatomic,14.0,20.0,0.00009,nonmetal,1766,1,1
1,2,He,Helium,4.002602,#D9FFFF,1s2,NaN,32.0,NaN,140.0,...,0.0,gas,atomic,NaN,4.0,0.00000,noble gas,1868,18,1
2,3,Li,Lithium,6.941,#CC80FF,[He] 2s1,0.98,134.0,76 (+1),182.0,...,-60.0,solid,metallic,454.0,1615.0,0.54000,alkali metal,1817,1,2
3,4,Be,Beryllium,9.012182,#C2FF00,[He] 2s2,1.57,90.0,45 (+2),NaN,...,0.0,solid,metallic,1560.0,2743.0,1.85000,alkaline earth metal,1798,2,2
4,5,B,Boron,10.811,#FFB5B5,[He] 2s2 2p1,2.04,82.0,27 (+3),NaN,...,-27.0,solid,covalent network,2348.0,4273.0,2.46000,metalloid,1807,13,2
5,6,C,Carbon,12.0107,#909090,[He] 2s2 2p2,2.55,77.0,16 (+4),170.0,...,-154.0,solid,covalent network,3823.0,4300.0,2.26000,nonmetal,Ancient,14,2
6,7,N,Nitrogen,14.0067,#3050F8,[He] 2s2 2p3,3.04,75.0,146 (-3),155.0,...,-7.0,gas,diatomic,63.0,77.0,0.00000,nonmetal,1772,15,2
7,8,O,Oxygen,15.9994,#FF0D0D,[He] 2s2 2p4,3.44,73.0,140 (-2),152.0,...,-141.0,gas,diatomic,55.0,90.0,0.00000,nonmetal,1774,16,2
8,9,F,Fluorine,18.9984032,#90E050,[He] 2s2 2p5,3.98,71.0,133 (-1),147.0,...,-328.0,gas,atomic,54.0,85.0,0.00000,halogen,1670,17,2
9,10,Ne,Neon,20.1797,#B3E3F5,[He] 2s2 2p6,NaN,69.0,NaN,154.0,...,0.0,gas,atomic,25.0,27.0,0.00000,noble gas,1898,18,2


In [21]:
#def create_scatter_from_df(dframe):
Title = "DataFrame FSR"
TOOLS = "hover, pan, wheel_zoom, box_zoom, reset, save"

p = figure(tools=TOOLS, toolbar_location="above", plot_width=1200, title=TITLE)
p.toolbar.logo = "grey"
p.background_fill_color = "#dddddd"
p.xaxis.axis_label = "Air Vel [cm/s]"
p.yaxis.axis_label = "FSR [mm/s]"
p.grid.grid_line_color = "white"
p.hover.tooltips = [
    ("core diameter", "@{d_c [mm]}"),
    ("insulator diameter", "@{d_i [mm]}"),
    ("Pressure (atm)", "@P_atm"),
    ("O2", "@[O2]"),
    ("external heat", "@{q_ext [kW/m^2]}")
]

source = ColumnDataSource(data)

p.circle("Air Vel [cm/s]", "FSR [mm/s]", size=12, source=source,
        line_color="black", fill_alpha=0.8)

labels = LabelSet(x="Air Vel [cm/s]", y="FSR [mm/s]", text="P_atm", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

output_file("FSR.html", title="FSR.py example")

show(p)

In [17]:
import pandas as pd

from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.sampledata.periodic_table import elements

elements = elements.copy()
elements = elements[elements["atomic number"] <= 82]
elements = elements[~pd.isnull(elements["melting point"])]
mass = [float(x.strip("[]")) for x in elements["atomic mass"]]
elements["atomic mass"] = mass

palette = ["#053061", "#2166ac", "#4393c3", "#92c5de", "#d1e5f0",
           "#f7f7f7", "#fddbc7", "#f4a582", "#d6604d", "#b2182b", "#67001f"]

melting_points = elements["melting point"]
low = min(melting_points)
high = max(melting_points)
melting_point_inds = [int(10*(x-low)/(high-low)) for x in melting_points] #gives items in colors a value from 0-10
elements['melting_colors'] = [palette[i] for i in melting_point_inds]

TITLE = "Density vs Atomic Weight of Elements (colored by melting point)"
TOOLS = "hover,pan,wheel_zoom,box_zoom,reset,save"

p = figure(tools=TOOLS, toolbar_location="above", plot_width=1200, title=TITLE)
p.toolbar.logo = "grey"
p.background_fill_color = "#dddddd"
p.xaxis.axis_label = "atomic weight (amu)"
p.yaxis.axis_label = "density (g/cm^3)"
p.grid.grid_line_color = "white"
p.hover.tooltips = [
    ("name", "@name"),
    ("symbol:", "@symbol"),
    ("density", "@density"),
    ("atomic weight", "@{atomic mass}"),
    ("melting point", "@{melting point}")
]

source = ColumnDataSource(elements)

p.circle("atomic mass", "density", size=12, source=source,
         color='melting_colors', line_color="black", fill_alpha=0.8)

labels = LabelSet(x="atomic mass", y="density", text="symbol", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

output_file("elements.html", title="elements.py example")

show(p)

In [7]:
show(p)

In [12]:
data

,Air Vel [cm/s],FSR [mm/s],d_c [mm],d_i [mm],P_atm,[O2],g,q_ext [kW/m^2],core material,insulation material,orientation (vert/horiz),up | down,Human Src ID,DOI,Plot #
0,0.134322,0.522128,1.0,3.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban,123123,0
1,14.111400,0.450124,1.0,3.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban,123123,0
2,23.839900,0.425121,1.0,3.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban,123123,0
3,31.571900,0.409509,1.0,3.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban,123123,0
4,41.551700,0.375101,1.0,3.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban,123123,0
5,54.019200,0.365802,1.0,3.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban,123123,0
6,65.975200,0.414513,1.0,3.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban,123123,0
7,75.700900,0.402054,1.0,3.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban,123123,0
8,99.881300,0.413240,1.0,3.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban,123123,0
9,125.329000,0.331929,1.0,3.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban,123123,0
